In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import pearsonr

#### 1. Reading Data

In [2]:
db_aq_20 = pd.read_excel('data/dublin_air_2020.xlsx')
db_tr_20 = pd.read_excel('data/dublin_traffic_2020.xlsx')

In [3]:
db_aq_20[' dateObserved__value_']

0       2020-01-01 00:00:00 +01:00/2020-01-01 01:00:00...
1       2020-01-01 01:00:00 +01:00/2020-01-01 02:00:00...
2       2020-01-01 02:00:00 +01:00/2020-01-01 03:00:00...
3       2020-01-01 03:00:00 +01:00/2020-01-01 04:00:00...
4       2020-01-01 04:00:00 +01:00/2020-01-01 05:00:00...
                              ...                        
8779    2020-12-31 19:00:00 +01:00/2020-12-31 20:00:00...
8780    2020-12-31 20:00:00 +01:00/2020-12-31 21:00:00...
8781    2020-12-31 21:00:00 +01:00/2020-12-31 22:00:00...
8782    2020-12-31 22:00:00 +01:00/2020-12-31 23:00:00...
8783    2020-12-31 23:00:00 +01:00/2021-01-01 00:00:00...
Name:  dateObserved__value_, Length: 8784, dtype: object

#### 2. Selecting Required Columns and Rows

In [5]:
# Selecting the specified columns and first 3648 rows (until 31.05.2020)
dfa = db_aq_20[['CO__value_','NO2__value_',' SO2__value_']].iloc[:3648,:]
dfa

,CO__value_,NO2__value_,SO2__value_
0,0.50000,NaN,NaN
1,0.53824,37.8,2.92699
2,0.40368,32.1,2.39481
3,0.53824,20.8,2.39481
4,0.53824,31.7,2.12872
...,...,...,...
3643,0.26912,22.9,7.18443
3644,0.26912,12.2,7.45052
3645,0.26912,29.2,8.24879
3646,0.40368,39.7,7.18443


In [6]:
# Selecting the specified columns and first 152 rows (until 31 May)
dft = db_tr_20[['SumOfSumVolume']].iloc[:152,:]
dft

,SumOfSumVolume
0,13810
1,19016
2,22224
3,19145
4,15802
...,...
147,14690
148,14175
149,15267
150,10856


#### 3. Checking and filling the missing values

In [7]:
#Checking missing values
dfa.isnull().sum()

CO__value_       8
NO2__value_     12
 SO2__value_    13
dtype: int64

In [8]:
#Checking missing values
dft.isnull().sum()

SumOfSumVolume    0
dtype: int64

In [9]:
# Filling missing values with 0
dfa1 = dfa.fillna(0);
dft1 = dft.fillna(0);

#### 4. Groupby 24 for Air Quality Data

In [10]:
# Groupby 24, 24 hours = 1 day for air quality data only
N = 24
dfa2 = dfa1.groupby(np.arange(len(dfa1.index))//N, axis=0).mean()
dfa2

/var/folders/nl/1bxm2b0s3j7f15g_30xsck8r0000gn/T/ipykernel_41874/3731954039.py:3: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  dfa2 = dfa1.groupby(np.arange(len(dfa1.index))//N, axis=0).mean()


,CO__value_,NO2__value_,SO2__value_
0,0.699240,38.225000,4.190917
1,0.459747,25.558333,3.935915
2,0.409287,23.641667,1.663062
3,0.409287,12.125000,1.541105
4,0.403680,7.950000,1.574366
...,...,...,...
147,0.269120,19.141667,2.317200
148,0.308367,23.904167,3.991350
149,0.285940,18.683333,4.202005
150,0.280333,24.658333,6.086809


#### 5. Calculating Correlation Coefficient

In [11]:
#pearsonr(dfa2['CO__value_'], dft1['SumOfSumVolume'])

In [12]:
dft1.corrwith(dfa2['CO__value_'], axis = 0)

SumOfSumVolume    0.063467
dtype: float64

In [13]:
dft1.corrwith(dfa2['NO2__value_'], axis = 0)

SumOfSumVolume    0.353524
dtype: float64

In [14]:
dft1.corrwith(dfa2[' SO2__value_'], axis = 0)

SumOfSumVolume    0.146764
dtype: float64

#### 6. Calculating P value between Traffic and Air Quality

In [15]:
# ttest_ind(dfa2['CO__value_'], dft1, equal_var=False).pvalue

In [16]:
ttest_ind(dft1, dfa2, equal_var= False).pvalue

array([4.22862690e-53, 4.76389489e-53, 4.34827624e-53])